In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [ ]:
U_TKN = '<usr>'
S_TKN = '<sys>'
BOS = '<bos>'
EOS = '<eos>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'
SEP = '<sep>'

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                          bos_token=BOS, eos_token=EOS, unk_token='<unk>',
                                          pad_token=PAD, mask_token=MASK, sep_token=SEP)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.bos_token

'<bos>'

In [ ]:
import pandas as pd

total_dataset_path = "/content/drive/MyDrive/3IS/total.csv"
df= pd.read_csv(total_dataset_path)

In [ ]:
df.head()

,question,answer,label
0,처음부터 서로 끌렸던 거 같아요.,그래서 어떻게 되었나요?,1
1,진짜 갑갑해서 미칠 것 같아요.,그랬군요. 제가 당신의 마음을 조금이라도 이해할 수 있으면 좋겠어요.,1
2,낚시 안 해봤는데 재미있어 보인다,도전해 봐도 좋을 거 같아요.,0
3,앞으로 집은 어떻게 사나 싶어요.,계속 앞으로 가다 보면 당신에게 맞는 길도 찾을 수 있지 않을까요?,1
4,첫사랑도 내가 보고 싶을까?,추억으로 남았다면 보고 싶고 궁금할거 같아요.,2


In [ ]:
df = df.drop(['label'], axis=1); df

,question,answer
0,처음부터 서로 끌렸던 거 같아요.,그래서 어떻게 되었나요?
1,진짜 갑갑해서 미칠 것 같아요.,그랬군요. 제가 당신의 마음을 조금이라도 이해할 수 있으면 좋겠어요.
2,낚시 안 해봤는데 재미있어 보인다,도전해 봐도 좋을 거 같아요.
3,앞으로 집은 어떻게 사나 싶어요.,계속 앞으로 가다 보면 당신에게 맞는 길도 찾을 수 있지 않을까요?
4,첫사랑도 내가 보고 싶을까?,추억으로 남았다면 보고 싶고 궁금할거 같아요.
...,...,...
17480,안 그러면 진짜 쓰러지거나 기절이라도 할 것 같았거든요.,그 상황이 되면 정말 아찔하죠. 지금은 괜찮으세요?
17481,몸이 아파서 하루에 두 세시간밖에 못 자.,저는 너무 잠이 오지 않을 때 아예 일어나서 책을 읽어요. 다른 일을 해보는 건 어...
17482,남편 벌어오는 돈도 시원찮고...,울적한 기분이 들면 저에게 털어버리세요.
17483,헤어진 사람과 같이 갔던 콘서트 영상이 티비에 나왔어,많은 생각이 들겠네요.


In [ ]:
conversations = df.values.tolist(); conversations[:5]

[['처음부터 서로 끌렸던 거 같아요.', '그래서 어떻게 되었나요?'],
 ['진짜 갑갑해서 미칠 것 같아요.', '그랬군요. 제가 당신의 마음을 조금이라도 이해할 수 있으면 좋겠어요.'],
 ['낚시 안 해봤는데 재미있어 보인다', '도전해 봐도 좋을 거 같아요.'],
 ['앞으로 집은 어떻게 사나 싶어요.', '계속 앞으로 가다 보면 당신에게 맞는 길도 찾을 수 있지 않을까요?'],
 ['첫사랑도 내가 보고 싶을까?', '추억으로 남았다면 보고 싶고 궁금할거 같아요.']]

In [ ]:
count = 0
class ChatDataset(Dataset):
    def __init__(self, conversations, tokenizer, max_length=150):
        self.tokenizer = tokenizer
        self.inputs = []
        self.labels = []

        BOS_ID = tokenizer.bos_token_id
        EOS_ID = tokenizer.eos_token_id
        SEP_ID = tokenizer.sep_token_id
        PAD_ID = tokenizer.pad_token_id

        for conversation in conversations:
            question, answer = conversation

            # 토큰화
            question_ids = tokenizer.encode(question)
            answer_ids = tokenizer.encode(answer)

            # [BOS] 질문 [SEP] 답변 [EOS] 형태로 만듬
            input_ids = [BOS_ID] + question_ids + [SEP_ID] + answer_ids + [EOS_ID]

            # 레이블 생성 (-100으로 패딩)
            label_ids = [-100]*(len(question_ids)+2) + answer_ids + [-100]*1

            # 패딩
            while len(input_ids) < max_length:
                input_ids.append(PAD_ID)
                label_ids.append(-100)

            self.inputs.append(input_ids)
            self.labels.append(label_ids)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx]), torch.tensor(self.labels[idx])

In [ ]:
# 토크나이저와 모델 로드
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
model.resize_token_embeddings(len(tokenizer))

# 데이터셋과 데이터 로더 준비
dataset = ChatDataset(conversations, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm import tqdm

# 훈련 루프
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(30):  # 에폭 수를 적절하게 조정해주세요.
    total_loss = 0
    for i, (inputs, labels) in enumerate(tqdm(dataloader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(input_ids=inputs, labels=labels)

        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {total_loss/(i+1)}")

    if epoch == 15:  # 15번째 epoch에서 모델 저장
        torch.save(model.state_dict(), "/content/drive/MyDrive/3IS/koGpt2_chatbot_epoch_15.pt")

torch.save(model.state_dict(), "/content/drive/MyDrive/3IS/koGpt2_chatbot_epoch_30.pt")

100%|██████████| 4372/4372 [14:11<00:00,  5.13it/s]


Epoch 1, Loss: 0.35909435076746415


100%|██████████| 4372/4372 [14:11<00:00,  5.14it/s]


Epoch 2, Loss: 0.22268151391377675


100%|██████████| 4372/4372 [14:11<00:00,  5.13it/s]


...

100%|██████████| 4372/4372 [14:11<00:00,  5.13it/s]


Epoch 21, Loss: 0.07268287309792983


100%|██████████| 4372/4372 [14:11<00:00,  5.13it/s]


Epoch 22, Loss: 0.06977048348222177


 48%|████▊     | 2114/4372 [06:51<07:20,  5.13it/s]

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/3IS/koGpt2_chatbot_epoch_15.pt", map_location=torch.device('cpu')))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# 질문을 입력받음
question = "배고프다."

# 토크나이징
inputs = tokenizer.encode(question, return_tensors='pt').to(device)

# 답변 생성
generated_high_temp = model.generate(inputs,
                           max_length=100,
                           temperature=3000.0,
                           no_repeat_ngram_size=1,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id
                           )

generated_low_temp = model.generate(inputs,
                           max_length=100,
                           temperature=0.1,
                           no_repeat_ngram_size=1,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id
                           )

print(generated_high_temp)
print(generated_low_temp)

# 텍스트로 변환
generated_text_high_temp = tokenizer.decode(generated_high_temp[:, inputs.shape[-1]:][0], skip_special_tokens=True)
generated_text_low_temp = tokenizer.decode(generated_low_temp[:, inputs.shape[-1]:][0], skip_special_tokens=True)

print(f'input: {question}')
print(f'output_high_temp: {generated_text_high_temp}')
print(f'output_low_temp: {generated_text_low_temp}')